In [118]:
import pandas as pd
import numpy as np
import tpqoa
from datetime import datetime, timedelta
import time
import pytz
from IPython.display import display, clear_output
import time
import sys
import matplotlib.pyplot as plt
plt.style.use("seaborn")

In [252]:
class BollTrader(tpqoa.tpqoa):
    def __init__(self, conf_file, instrument, bar_length, SMA, dev, units):
        super().__init__(conf_file)
        self.instrument = instrument
        self.bar_length = pd.to_timedelta(bar_length)
        self.tick_data = pd.DataFrame()
        self.raw_data = None
        self.data = None 
        self.last_bar = None
        self.units = units
        self.position = 0
        self.profits = []
        
        #*****************add strategy-specific attributes here******************
        self.SMA = SMA
        self.dev = dev
        #************************************************************************
    
    def get_most_recent(self, days = 5):
        while True:
            time.sleep(2)
            now = datetime.utcnow()
            now = now - timedelta(microseconds = now.microsecond)
            past = now - timedelta(days = days)
            df = self.get_history(instrument = self.instrument, start = past, end = now,
                                   granularity = "S5", price = "M", localize = False).c.dropna().to_frame()
            df.rename(columns = {"c":self.instrument}, inplace = True)
            df = df.resample(self.bar_length, label = "right").last().dropna().iloc[:-1]
            self.raw_data = df.copy()
            self.last_bar = self.raw_data.index[-1]
            if pd.to_datetime(datetime.utcnow()).tz_localize("UTC") - self.last_bar < self.bar_length:
                break
                
    def on_success(self, time, bid, ask):
        if self.ticks > 1:
            sys.stdout.write(str(self.tick_data.tail(1))+"\n\n\n")
        
        recent_tick = pd.to_datetime(time)
        df = pd.DataFrame({self.instrument:(ask + bid)/2}, 
                          index = [recent_tick])
        self.tick_data = self.tick_data.append(df)
        
        if recent_tick - self.last_bar > self.bar_length:
            self.resample_and_join()
            self.define_strategy()
            self.execute_trades()
    
    def resample_and_join(self):
        self.raw_data = self.raw_data.append(self.tick_data.resample(self.bar_length, 
                                                                  label="right").last().ffill().iloc[:-1])
        self.tick_data = self.tick_data.iloc[-1:]
        self.last_bar = self.raw_data.index[-1]
        
    
    def define_strategy(self): # "strategy-specific"
        df = self.raw_data.copy()
        
        #******************** define your strategy here ************************
        df["SMA"] = df[self.instrument].rolling(self.SMA).mean()
        df["Lower"] = df["SMA"] - df[self.instrument].rolling(self.SMA).std() * self.dev
        df["Upper"] = df["SMA"] + df[self.instrument].rolling(self.SMA).std() * self.dev
        df["distance"] = df[self.instrument] - df.SMA
        df["position"] = np.where(df[self.instrument] < df.Lower, 1, np.nan)
        df["position"] = np.where(df[self.instrument] > df.Upper, -1, df["position"])
        df["position"] = np.where(df.distance * df.distance.shift(1) < 0, 0, df["position"])
        df["position"] = df.position.ffill().fillna(0)
        #***********************************************************************
        
        self.data = df.copy()
    
    def execute_trades(self):
        sys.stdout.write("\n")
        if self.data["position"].iloc[-1] == 1:
            if self.position == 0:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING LONG")  # NEW
            elif self.position == -1:
                order = self.create_order(self.instrument, self.units * 2, suppress = True, ret = True) 
                self.report_trade(order, "GOING LONG")  # NEW
            self.position = 1
        elif self.data["position"].iloc[-1] == -1: 
            if self.position == 0:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  # NEW
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units * 2, suppress = True, ret = True)
                self.report_trade(order, "GOING SHORT")  # NEW
            self.position = -1
        elif self.data["position"].iloc[-1] == 0: 
            if self.position == -1:
                order = self.create_order(self.instrument, self.units, suppress = True, ret = True) 
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            elif self.position == 1:
                order = self.create_order(self.instrument, -self.units, suppress = True, ret = True)
                self.report_trade(order, "GOING NEUTRAL")  # NEW
            self.position = 0
    
    def report_trade(self, order, going):
        time = order["time"]
        units = order["units"]
        price = order["price"]
        pl = float(order["pl"])
        self.profits.append(pl)
        cumpl = sum(self.profits)
        sys.stdout.write(110* "-"+"\n")
        sys.stdout.write("{} | {}\n\n".format(time, going))
        sys.stdout.write("{} | units = {} | price = {} | P&L = {} | Cum P&L = {}\n".format(time, units, price, pl, cumpl))
        sys.stdout.write(110 * "-" + "\n")

In [253]:
test_trader = BollTrader('oanda.cfg', "EUR_USD", "1min", SMA = 1, dev = 2, units = 200000)
test_trader.create_order(test_trader.instrument, 1, suppress = True, ret = True)

{'id': '541',
 'time': '2022-06-10T04:27:51.225543409Z',
 'userID': 22130075,
 'accountID': '101-001-22130075-001',
 'batchID': '540',
 'requestID': '24955280707675030',
 'type': 'ORDER_FILL',
 'orderID': '540',
 'instrument': 'EUR_USD',
 'units': '1.0',
 'gainQuoteHomeConversionFactor': '1.0',
 'lossQuoteHomeConversionFactor': '1.0',
 'price': 1.06322,
 'fullVWAP': 1.06322,
 'fullPrice': {'type': 'PRICE',
  'bids': [{'price': 1.06308, 'liquidity': '10000000'}],
  'asks': [{'price': 1.06322, 'liquidity': '10000000'}],
  'closeoutBid': 1.06308,
  'closeoutAsk': 1.06322},
 'reason': 'MARKET_ORDER',
 'pl': '0.0',
 'financing': '0.0',
 'commission': '0.0',
 'guaranteedExecutionFee': '0.0',
 'accountBalance': '9256.9841',
 'tradeOpened': {'tradeID': '541',
  'units': '1.0',
  'price': 1.06322,
  'guaranteedExecutionFee': '0.0',
  'halfSpreadCost': '0.0001',
  'initialMarginRequired': '0.0354'},
 'halfSpreadCost': '0.0001'}

In [254]:
test_trader.get_most_recent()
test_trader.stream_data(test_trader.instrument, ret=True, stop = 5)
if test_trader.position != 0: # if we have a final open position
    close_order = test_trader.create_order(test_trader.instrument,\
    units = -test_trader.position * test_trader.units, suppress = True, ret = True) 
    test_trader.report_trade(close_order, "GOING NEUTRAL")
    test_trader.position = 0


                                     EUR_USD
2022-06-10 04:28:06.338389927+00:00  1.06313


                                      EUR_USD
2022-06-10 04:28:12.940643708+00:00  1.063135


                                      EUR_USD
2022-06-10 04:28:13.919332109+00:00  1.063135


                                     EUR_USD
2022-06-10 04:28:16.702160692+00:00  1.06313




In [251]:
print(test_trader.tick_data)

                                      EUR_USD
2022-06-10 04:25:30.327686088+00:00  1.063295
2022-06-10 04:25:35.543825784+00:00  1.063320
2022-06-10 04:25:35.685653422+00:00  1.063290
2022-06-10 04:25:36.061078717+00:00  1.063295
2022-06-10 04:25:36.154227599+00:00  1.063290


In [106]:
paper_EURUSD = BollTrader('oanda.cfg', "EUR_USD", "361min", SMA = 49, dev = 1.7058823529411764, units = 200000)
paper_EURUSD.get_most_recent()
paper_EURUSD.stream_data(paper_EURUSD.instrument, stop = 20)
if trader.position != 0: # if we have a final open position
    close_order = paper_EURUSD.create_order(paper_EURUSD.instrument,\
    units = -paper_EURUSD.position * paper_EURUSD.units, suppress = True, ret = True) 
    paper_EURUSD.report_trade(close_order, "GOING NEUTRAL")
    paper_EURUSD.position = 0

1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 19 20 

In [109]:
trader.profits

[0.0, 0.0, 0.0, -28.0]

In [124]:
sys.stdout.write('hello'+100*"8")

hello8888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888888

105

In [255]:
net_units = 0
trx = trader.get_transactions()
for i in range(len(trx)):
    if trx[i]['type'] == 'MARKET_ORDER' or trx[i]['type'] == 'LIMIT_ORDER':
        print(trx[i]['time'], trx[i]['instrument'], trx[i]['units'], sep = " | ", end = "\n")
        net_units =+ float(trx[i]['units'])
#trx
net_units
trader.get_account_summary()

2022-04-10T20:32:07.453952224Z | EUR_USD | 100000.0
2022-04-10T20:32:20.609012414Z | EUR_USD | 100000.0
2022-04-10T20:32:32.576752389Z | EUR_USD | 100000.0
2022-04-10T20:32:40.006626143Z | EUR_USD | 100000.0
2022-04-10T20:32:46.136026021Z | EUR_USD | 100000.0
2022-04-10T21:25:14.593036488Z | EUR_USD | 100000.0
2022-04-10T22:04:33.115629037Z | EUR_USD | -100000.0
2022-04-10T22:09:26.665926151Z | EUR_USD | 100000.0
2022-04-10T22:30:52.195037392Z | EUR_USD | -100000.0
2022-04-11T00:17:10.043743507Z | EUR_USD | 10000.0
2022-04-11T00:17:18.033491823Z | EUR_USD | -10000.0
2022-04-11T00:19:45.414852790Z | EUR_USD | 50000.0
2022-04-11T12:07:14.554584133Z | EUR_USD | 10000.0
2022-04-11T12:07:26.808848247Z | EUR_USD | -50000.0
2022-04-13T03:05:16.862831526Z | EUR_USD | 100000.0
2022-04-13T03:11:27.050108829Z | EUR_USD | 100000.0
2022-04-13T03:11:43.479753343Z | EUR_USD | 100000.0
2022-04-13T03:11:54.679791801Z | EUR_USD | -100000.0
2022-04-13T03:11:59.104172096Z | EUR_USD | 100000.0
2022-04-13T0

{'id': '101-001-22130075-001',
 'alias': 'Primary',
 'currency': 'USD',
 'balance': '10000.004',
 'createdByUserID': 22130075,
 'createdTime': '2022-04-10T20:08:48.158233392Z',
 'guaranteedStopLossOrderMode': 'DISABLED',
 'pl': '-1130.8757',
 'resettablePL': '-1130.8757',
 'resettablePLTime': '0',
 'financing': '-1.2403',
 'commission': '0.0',
 'guaranteedExecutionFees': '0.0',
 'marginRate': '0.0333',
 'openTradeCount': 0,
 'openPositionCount': 0,
 'pendingOrderCount': 0,
 'hedgingEnabled': False,
 'unrealizedPL': '0.0',
 'NAV': '10000.004',
 'marginUsed': '0.0',
 'marginAvailable': '10000.004',
 'positionValue': '0.0',
 'marginCloseoutUnrealizedPL': '0.0',
 'marginCloseoutNAV': '10000.004',
 'marginCloseoutMarginUsed': '0.0',
 'marginCloseoutPercent': '0.0',
 'marginCloseoutPositionValue': '0.0',
 'withdrawalLimit': '10000.004',
 'marginCallMarginUsed': '0.0',
 'marginCallPercent': '0.0',
 'lastTransactionID': '548'}